In [1]:
import sys
sys.path.append("../../src")

import numpy as np
import pandas as pd

import lightgbm

from utils import ndcg_calculator
from LGBM_Rank import LGBMRank
from dataload import dataload, day_week_feature_engineering, train_label_split, feature_dataload
from feature_engineering import *
from candidate import *
from tqdm import tqdm

../../src/dataload.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def day_feature(df_train:pd.DataFrame())->pd.Series():


# dataload
- 편의상 df_train_week 변수 통일하여 사용함

In [2]:
path='../../data/'

test_answer_week, test_answer_month, \
df_train_week, df_train_month, \
sample_sumbission_week, sample_sumbission_month = dataload(path)

In [3]:
# # # 전체 데이터 이용시
# df_history = pd.read_csv(path+'history_data.csv')

# # ## 날짜 전처리
# df_history = df_history.assign(log_dt = pd.to_datetime(df_history.log_time//100, format="%Y%m%d%H%M"))
# df_history = df_history.assign(log_date = df_history.log_dt.dt.floor("D"))
# df_history = df_history.drop("log_time", axis=1)

# df_train_week = df_history.copy()

# # # month 이용
# # # df_train_week = df_train_month.copy()

# feature dataload

In [4]:
# data load
watch_df, buy_df, search_df, meta_df, profile_df = feature_dataload(path)

../../src/dataload.py:87: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(path + "meta_data.csv")


In [5]:
# feature engineering
# df_train_week = history_feature_engineering(df_train_week)

# short_con_freq_feature = df_train_week[["album_id","short_trailer","continuous_play","album_viewcount_freq"]]

watch_df_feature = watch_feature_engineering(watch_df)
watch_df_feature = watch_df_feature.drop_duplicates(subset=["profile_id","album_id"])

paid_df_feature = paid_feature_engineering(df_train_week, buy_df)

searched_df_feature = searched_feature_engineering(df_train_week, search_df)

meta_df_feature = meta_feature_engineering(meta_df)
meta_df_feature = meta_df_feature.drop(columns="sub_title").drop_duplicates()

profile_df_feature = profile_feature_engineering(profile_df)
profile_df_feature = profile_df_feature[["profile_id","sex","age","age_bin"]]

# day_week feature engineering
- to split label, train data

In [6]:
df_train_week = day_week_feature_engineering(df_train_week)

log date min: 2022-03-01 00:00:00 log date max: 2022-04-23 00:00:00
min day: 0 max day: 53
min week: 0 max week: 7


# label & train split

In [7]:
# Label, train data split
df_train, label_df = train_label_split(df_train_week, cv_date=0)

split last week: 7


In [8]:
n = 25

In [9]:
len(df_train), len(df_train_week)

(945518, 945518)

# prerpocess

In [10]:
# 총 3개 파일 사용

# log dt 기준 duplicates
df_train_week = df_train.drop_duplicates(subset=["profile_id","album_id","log_dt"])
# ss_id 기준 duplicates
personal_train = df_train.drop_duplicates(subset=['profile_id','album_id','ss_id'])
# album_id, profile_id 기준으로 duplicates
df_train = df_train.drop_duplicates(subset=['profile_id','album_id'])

# 전체 유저 한정
customers = df_train.profile_id.unique()
# 라벨 한정
# customers = label_df.profile_id.unique()

# day week features

In [11]:
interaction_day_week_first = day_week_feature(df_train_week)
hour_feature = hour_feature(df_train_week)

../../src/feature_engineering.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hour_feature["hour"] = hour_feature["log_dt"].apply(lambda x: x.hour)


# candidate generation

# MP

In [12]:
MP_cand, MP_feature = most_popular(df_train)

# general MP
- 마지막 1주, 2주의 MP를 각 유저마다 넣는다.

In [13]:
%%time
general_MP_cand, general_MP_feature = general_most_popular(df_train)

CPU times: user 60.5 ms, sys: 13.3 ms, total: 73.8 ms
Wall time: 71.5 ms


# personal_MP

In [14]:
%%time
personal_MP_candidate, personal_MP_feature = personal_most_popular(personal_train, N=5)

100%|█████████████████████████████████████| 8057/8057 [00:05<00:00, 1484.20it/s]


CPU times: user 5.71 s, sys: 112 ms, total: 5.82 s
Wall time: 5.8 s


# MP_user_genre

In [15]:
# 이전에는 meta 중복 때문에 장르 count가 잘못 뻥튀기 되어서 잘못 MP가 매겨짐
# feature merge 시 genre_mid 와 profile_id 기준으로 해야함
genre_candidate, user_genre_feature = user_genre_most_popular(personal_train, meta_df)

# ALS MF candidate

In [16]:
%%time
from ALS_MF import MF
model = MF(df_train_week, clf={'factors': 200, 'regularization': 0.005, 'iterations': 3, 'n': 100})

user_factors_feature = pd.read_parquet(path+"profile_factor_als.parquet")
item_factors_feature = pd.read_parquet(path+"album_factor_als.parquet")

als_candidate, _, _, als_model = model.mf_train()

  0%|          | 0/3 [00:00<?, ?it/s]

item vector shape: (20383, 200) user vector shape (8057, 200)
CPU times: user 13 s, sys: 5.26 s, total: 18.2 s
Wall time: 13.9 s


# apriori candidaate

In [17]:
%%time
from apriori import apriori_train, apriori_candidate
rules_confidence_item_week = apriori_train(df_train, 0.1, 0.8)
apriori_candidate, apriori_feature = apriori_candidate(df_train_week, rules_confidence_item_week)

CPU times: user 2.75 s, sys: 56 ms, total: 2.81 s
Wall time: 2.81 s


# NCF candidate

In [18]:
ncf_candidate_df = pd.read_csv(path + "2_neg100_submission.csv")

In [19]:
import ast

ncf_candidate_list = []
for idx, row in ncf_candidate_df.iterrows():
    ncf_df = pd.DataFrame()
    ncf_df["album_id"] = ast.literal_eval(row.predicted_list)
    ncf_df["profile_id"] = row.profile_id
    ncf_candidate_list.append(ncf_df)
ncf_candidate = pd.concat(ncf_candidate_list)
ncf_candidate = ncf_candidate[ncf_candidate["profile_id"].isin(set(df_train.profile_id))]

# 아웃라이어 삭제
album_max_dict = df_train.groupby("profile_id")["album_id"].max().to_dict()

last_train_df_list = []
for user_id in ncf_candidate.profile_id.unique():
    last_train_df_list.append(ncf_candidate[(ncf_candidate["profile_id"]==user_id)&(ncf_candidate["album_id"]<=album_max_dict[user_id])])
ncf_candidate = pd.concat(last_train_df_list)

# NCG 모델만 테스트
# ncf_test_answer_week = ncf_candidate.groupby("profile_id")["album_id"].unique().reset_index()
# ncf_test_answer_week = ncf_test_answer_week[ncf_test_answer_week["profile_id"].isin(set(sample_sumbission_week.profile_id))]
# ncf_test_answer_week = ncf_test_answer_week.reset_index(drop=True)

# ndcg_calculator(test_answer_week, ncf_test_answer_week, 25)

# series candidate

In [20]:
series_train_week= df_train_week.copy()
tmp = series_train_week.groupby('profile_id').log_date.max().reset_index()
tmp.columns = ['profile_id','max_dat']
tmp.rename(columns = {'log_date':'max_dat'},inplace=True)
series_train_week = series_train_week.merge(tmp,on=['profile_id'],how='left')
series_train_week['diff_dat'] = (series_train_week.max_dat - series_train_week.log_date).dt.days 
series_train_week = series_train_week.loc[series_train_week['diff_dat']<=6] 
print('Train shape:',series_train_week.shape)

Train shape: (156060, 13)


In [21]:
tmp = series_train_week.groupby(['profile_id','album_id'])['log_date'].agg('count').reset_index() 
tmp.columns = ['profile_id','album_id','ct'] 
series_train_week = series_train_week.merge(tmp,on=['profile_id','album_id'],how='left')  
series_train_week = series_train_week.sort_values(['ct','log_date'],ascending=False) 
series_train_week = series_train_week.drop_duplicates(['profile_id','album_id']) 
series_train_week = series_train_week.sort_values(['ct','log_date'],ascending=False) 

In [22]:
# 1. 영상 3개 이하 본 유저 리스트 / len(cold_user) : 670
user_list = series_train_week.profile_id.value_counts().reset_index()
user_list.columns = ['profile_id','view_count']
cold_user = user_list[user_list['view_count']<10].index.values.tolist()

In [23]:
#판매된 아이템 개수를, 아이템 ID별로 개수 집계
vc_ = series_train_week.album_id.value_counts().reset_index()
vc = vc_[vc_.album_id > 10].set_index('index')
pairs = {}
for j,i in tqdm(enumerate(vc.index.values)):
    USERS_ = series_train_week.loc[series_train_week.album_id==i.item(),'profile_id'].unique().tolist()
    USERS = [x for x in USERS_ if x not in cold_user]
    vc2 = series_train_week.loc[(series_train_week.profile_id.isin(USERS))&(series_train_week.album_id!=i.item()),'album_id'].value_counts()
    vc2 = vc2[vc2>=10]
    if len(vc2.index)< 4:
        pairs[i.item()] = [vc2.index[i] for i in range(len(vc2.index))]
    else:
        pairs[i.item()] = [vc2.index[0],vc2.index[1],vc2.index[2],vc2.index[3]]

1897it [00:05, 337.66it/s]


In [24]:
cnt = 0
for i in pairs.values():
    if len(i) >0:
        cnt+=1
print(cnt)

1251


In [25]:
df_list = []
for user_id in tqdm(df_train_week.profile_id.unique()):
    df = pd.DataFrame()
    album_list = []
    for album_id in df_train_week[df_train_week["profile_id"]==user_id].album_id.unique():
        album_list += pairs.get(album_id, [])
    df["album_id"] = album_list
    df["profile_id"] = user_id
    df_list.append(df)
series_candidate = pd.concat(df_list)
series_candidate["album_id"] = series_candidate["album_id"].astype(int)
print(series_candidate.profile_id.nunique())

100%|██████████████████████████████████████| 8057/8057 [00:09<00:00, 859.76it/s]


7650


# candidate merge

In [26]:
general_MP_cand.album_id.nunique(), personal_MP_candidate.album_id.nunique(), genre_candidate.album_id.nunique(),\
als_candidate.album_id.nunique(), apriori_candidate.album_id.nunique(), ncf_candidate.album_id.nunique(), series_candidate.album_id.nunique()

(17, 2146, 211, 8448, 4, 14939, 1177)

In [27]:
candidate_0 = MP_cand
candidate_1 = general_MP_cand[['profile_id','album_id']]
candidate_2 = personal_MP_candidate[['profile_id','album_id']]
candidate_3 = genre_candidate[['profile_id','album_id']]
candidate_4 = als_candidate.copy()
candidate_5 = apriori_candidate.copy()
candidate_6 = ncf_candidate.copy()
candidate_7 = series_candidate.copy()

candidate = [candidate_0, candidate_1, candidate_2, candidate_3, candidate_4,candidate_5, candidate_6,candidate_7 ]

cand = pd.concat(candidate)

cand.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

print('cand 데이터 수:',len(cand))
print('cand nunique: ', cand.album_id.nunique())

cand 데이터 수: 1290326
cand nunique:  15941


# features merge

In [28]:
candidate = pd.merge(cand, MP_feature, how='left', on='album_id')
# general counts feature merge
candidate = pd.merge(candidate, general_MP_feature, how='left', on='album_id')
# personal_count feature merge
candidate = pd.merge(candidate, personal_MP_feature, how='left', on=['profile_id','album_id'])
candidate["personal_counts"] = candidate["personal_counts"].fillna(1)
# meta feature
candidate = pd.merge(candidate, meta_df_feature, how="left",on="album_id")
# genre feature merge(genre_cnt, user_genre_total_cnt, user_genre_total_percent)
candidate = pd.merge(candidate, user_genre_feature, how='left', on=['profile_id','genre_mid'])
# apriori feature merge
candidate = pd.merge(candidate, apriori_feature, how="left", on=["profile_id","album_id"])

In [29]:
print('cand 피처 수:',len(cand))

cand 피처 수: 1290326


In [30]:
# meta feature merge

'''
아직 사용 안하고 있음
# profile feature add
candidate_add_features = pd.merge(candidate,profile_df, how='left', on='profile_id')
# meta feature add
candidate_add_features = pd.merge(candidate_add_features, meta_df.drop_duplicates('album_id'), how='left', on='album_id')

# catetegory feature label encoding

columns = ['sex','pr_interest_keyword_cd_1','pr_interest_keyword_cd_2','pr_interest_keyword_cd_3','ch_interest_keyword_cd_1','ch_interest_keyword_cd_2','ch_interest_keyword_cd_3',\
'genre_mid','cast_1','cast_2','cast_3']
from sklearn.preprocessing import LabelEncoder
for col in columns:
    LE = LabelEncoder()
    candidate_add_features[col] = LE.fit_transform(candidate_add_features[col])

# drop columns

candidate_add_features = candidate_add_features.drop(columns=['personal_counts','pr_interest_keyword_cd_1',\
                                                              'pr_interest_keyword_cd_2','pr_interest_keyword_cd_3'\
                                                              ,'ch_interest_keyword_cd_1','ch_interest_keyword_cd_2'\
                                                              ,'ch_interest_keyword_cd_3',\
                                                                'genre_mid','cast_1','cast_2','cast_3'])
                                                                
'''
candidate_add_features = candidate.copy()

# als vector merge
candidate_add_features = pd.merge(candidate_add_features, item_factors_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features, user_factors_feature, how="left", on="profile_id")

# interaction day, week merge
candidate_add_features = pd.merge(candidate_add_features, interaction_day_week_first, how="left",on=["profile_id","album_id"])

In [31]:
# short_con_freq_feature = short_con_freq_feature[["album_id","album_viewcount_freq"]].drop_duplicates()
# candidate_add_features = pd.merge(candidate_add_features, short_con_freq_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features, watch_df_feature, how="left",on=["profile_id","album_id"])
candidate_add_features = pd.merge(candidate_add_features, profile_df_feature, how="left",on="profile_id")
candidate_add_features = pd.merge(candidate_add_features, searched_df_feature,how="left",on=["profile_id","album_id"])
candidate_add_features = pd.merge(candidate_add_features, paid_df_feature,how="left",on=["profile_id","album_id"])
candidate_add_features["paid_label"] = candidate_add_features["paid_label"].fillna(0)
# 타입오류 픽처 제거
candidate_add_features = candidate_add_features.drop(columns=["onair_date","title", "genre_small", "cast_1", "cast_2", "cast_3", "cast_4", "cast_5", "cast_6", "cast_7"])

In [32]:
len(candidate_add_features)

1290326

In [33]:
# 라벨 부여
train_df = pd.merge(candidate_add_features, label_df, how='left', on=['profile_id','album_id'])
train_df['rating'] = train_df['rating'].fillna(0)

# 비복원 추출 defalut
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [34]:
train_df["rating"].value_counts()

0.0    1243610
1.0      46716
Name: rating, dtype: int64

# model

In [35]:
# # # feature selection 컬럼만 학습
# train_df = train_df[['profile_id','album_id','day', 'personal_counts', 'week', 'total_counts',
#        'item_vector66', 'user_vector51', 'user_vector119', 'item_vector76',
#        'user_vector59', 'item_vector97', 'user_vector164', 'user_vector196',
#        'user_vector178', 'user_vector147', 'user_vector99', 'user_vector167',
#        'item_vector165', 'user_vector75', 'user_vector50', 'user_vector136',
#        'user_vector108', 'user_vector131', 'user_vector18', 'user_vector49',
#        'user_vector62', 'user_vector152', 'user_vector177', 'item_vector119','rating']]

In [36]:
lgbmrank = LGBMRank(train_df, mode='week', model_params={'n_estimators':5})
X_train, sample_sumbission = lgbmrank.valid_evaluation()

                     feature_importances
total_counts                  892.383987
general_counts_ver1          2889.958008
general_counts_ver2            59.969002
personal_counts             75560.971838
genre_large                     0.000000
...                                  ...
sex                             0.000000
age                             0.000000
age_bin                         0.000000
searched_label                  0.000000
paid_label                      0.000000

[423 rows x 1 columns]
week performance
lgbm ndcg: 0.2014515343155575


In [ ]:
label_users = label_df.profile_id.unique()
label_train_df = train_df[train_df["profile_id"].isin(label_users)]

label_train_df["rating"].value_counts()

In [ ]:
lgbmrank = LGBMRank(label_train_df, mode='week', model_params={'n_estimators':5})
model, feature_importances_df = lgbmrank.train()
# X_train, sample_sumbission = lgbmrank.valid_evaluation()

In [ ]:
# 모델 앙상블 모델 파일 만들기
# import pickle

# file = 'lgb_model_ncf_3.pkl'
# pickle.dump(model, open(file, 'wb'))
# print('Trained LGB model was saved!')

# # load model
# import joblib
# lgb_model_1 = joblib.load('lgb_model_ncf_1.pkl')
# lgb_model_2 = joblib.load('lgb_model_ncf_2.pkl')
# lgb_model_3 = joblib.load('lgb_model_ncf_3.pkl')

In [ ]:
user_idx = train_df.profile_id
item_idx = train_df.album_id

del train_df["profile_id"], train_df["album_id"], train_df["rating"]

In [ ]:
pred = model.predict(train_df)
train_df["profile_id"] = user_idx
train_df["album_id"] = item_idx
train_df["pred"] = pred

lgbm_sub_df = train_df.sort_values(by='pred', ascending=False).groupby('profile_id').head(25)
lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
sample_sumbission_week['album_id'] = sample_sumbission_week['profile_id'].apply(lambda x: lgbm_user_items_dict.get(x, []))

print('lgbm ndcg:', ndcg_calculator(sample_sumbission_week, test_answer_week, 25))

# 최종 제출

In [ ]:
sample_submission_week = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
pred = model.predict(train_df)
train_df["profile_id"] = user_idx
train_df["album_id"] = item_idx
train_df["pred"] = pred

lgbm_sub_df = train_df.sort_values(by='pred', ascending=False).groupby('profile_id').head(25)
lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
sample_submission_week['predicted_list'] = sample_submission_week['profile_id'].apply(lambda x: list(lgbm_user_items_dict.get(x, [])))

# print('lgbm ndcg:', ndcg_calculator(sample_submission_week, test_answer_week, 25))

In [ ]:
sample_submission_week.to_csv("submission_ver7.csv",index=False)

# 모델 CV 부분

In [ ]:
# pred_1 = lgb_model_1.predict(train_df)
# pred_2 = lgb_model_2.predict(train_df)
# pred_3 = lgb_model_3.predict(train_df)

# pred = np.mean([pred_1, pred_2, pred_3], axis=0)
# train_df["profile_id"] = user_idx
# train_df["album_id"] = item_idx
# train_df["pred"] = pred

# lgbm_sub_df = train_df.sort_values(by='pred', ascending=False).groupby('profile_id').head(25)
# lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
# sample_sumbission_week['album_id'] = sample_sumbission_week['profile_id'].apply(lambda x: lgbm_user_items_dict.get(x, []))

# print('lgbm ndcg:', ndcg_calculator(sample_sumbission_week, test_answer_week, 25))

In [ ]:
def evaluation(model, train_df, df_train, sample_sumbission_week,MP_cand, user_idx, item_idx):
    pred = model.predict(train_df)
    train_df["profile_id"] = user_idx
    train_df["album_id"] = item_idx
    train_df["pred"] = pred

    album_max_dict = df_train.groupby("profile_id")["album_id"].max().to_dict()

    last_train_df_list = []
    for user_id in train_df.profile_id.unique():
        last_train_df_list.append(train_df[(train_df["profile_id"]==user_id)&(train_df["album_id"]<=album_max_dict[user_id])])
    after_train_df = pd.concat(last_train_df_list)

    lgbm_sub_df = after_train_df.sort_values(by='pred', ascending=False).groupby('profile_id').head(25)
    lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
    sample_sumbission_week['predicted_list'] = sample_sumbission_week['profile_id'].apply(lambda x: lgbm_user_items_dict.get(x, []))
    
#     MP_list = MP_cand.album_id.unique()
    
#     # cold start user file MP_list top25
#     sumbission_cold = sample_sumbission_week.copy()
#     sumbission_cold['predicted_list'] = sumbission_cold['predicted_list']\
#                                             .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])
    
#     print('lgbm ndcg:', ndcg_calculator(sample_sumbission_week, test_answer_week, 25))
#     return sumbission_cold
    return sample_sumbission_week, album_max_dict

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
sample_sumbission_week, album_max_dict = evaluation(model, train_df, df_train, submission, MP_cand, user_idx, item_idx)

for i in sample_sumbission_week.predicted_list:
    if len(i)<25:
        print(len(i))

In [ ]:
train_df.groupby("profile_id").count()["album_id"][train_df.groupby("profile_id").count()["album_id"]<25]

In [ ]:
sample_sumbission_week.to_csv("sumbission_ver12.csv",index=False)

In [ ]:
# meta_df = pd.read_csv(path+"meta_data.csv")
meta_df = meta_df.drop(columns=["genre_large","run_time","onair_date"])

In [ ]:
columns = ["title","sub_title","genre_mid","genre_small","country","cast_1","cast_2","cast_3","cast_4","cast_5","cast_6","cast_7"]
meta_one_hot_df = pd.get_dummies(meta_df, columns = columns)

In [ ]:
series_list = []
for idx, row in tqdm(meta_df.iterrows()):
    row_dict = {}
    row_dict["album_id"] = [row["album_id"]]
    for col in columns:
        if type(row[col]) == str:
            row_dict[row[col]] = [1]
    series_list.append(row_dict)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=lgbm_sub_df)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=df_train)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=label_df)
plt.show()

In [ ]:
real_target_list = []
for idx, row in test_answer_week.iterrows():
    df = pd.DataFrame()
    df["album_id"] = row["album_id"]
    df["profile_id"] = row["profile_id"]
    real_target_list.append(df)
real_target_df = pd.concat(real_target_list)
pred_df = lgbm_sub_df[["profile_id","album_id","pred"]]

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=pred_df)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,10))
sns.scatterplot(x="profile_id" , y="album_id" , data=real_target_df)
plt.show()

In [ ]:
# cold user MP로 채우기 
MP_list = MP_cand.album_id.unique()

sample_sumbission_cold = sample_sumbission.copy()
sample_sumbission_cold['album_id'] = sample_sumbission_cold['album_id']\
                                        .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])

In [ ]:
ndcg_calculator(sample_sumbission_cold, test_answer_week, n)

# Evaluation

### experiment
- num_leaves= 20, learning_rate=0.005, n_estimators:5
- 어느 cand 중요한 지 판단
- popular_articles_cand :(10,10) , personal_MP_candidate:(5), genre_candidate: max(10,10)

### week
- week ndcg score : 0.05611031122402796
- popular_articles_cand week: ndcg 0.028022766659891125 (total unique item: 12)
- personal_MP_candidate week: ndcg 0.02896244104871668  (total unique item: 4303)
- genre_candidate week:       ndcg 0.01579135161487282  (total unique item: 157)
- (popular_articles_cand, personal_MP_candidate) week: ndcg 0.05206722084683086  (total unique item: 4303)
- (popular_articles_cand, genre_candidate) week: 0.03809584406788012 (total unique item: 161)
- (personal_MP_candidate, genre_candidate) week: 0.041085241338965385 (total unique item: 4338)

### month
- month ndcg score : 0.08036130090004782
- popular_articles_cand month: ndcg 0.05945057771810242 (total unique item: 15)
- personal_MP_candidate month: ndcg 0.028477345633295483  (total unique item: 2826)
- genre_candidate       month: ndcg 0.017431233023063257  (total unique item: 150)
- (popular_articles_cand, personal_MP_candidate) month: ndcg 0.05206722084683086  (total unique item: 2826)
- (popular_articles_cand, genre_candidate) month: 0.0680887232688415 (total unique item: 155)
- (personal_MP_candidate, genre_candidate) month: 0.041085241338965385 (total unique item: 2872)

In [ ]:
def evaluation(
            X_train:pd.DataFrame(), 
            sumbission:pd.DataFrame(), 
            n:int, 
            MP_cand:pd.DataFrame()
            )->pd.DataFrame():
    
    MP_list = MP_cand.album_id.unique()
    
    # each user pred 25 items
    lgbm_sub_df = X_train.sort_values(by='pred', ascending=False).groupby('profile_id').head(n)
    lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
    sumbission['predicted_list'] = sumbission['profile_id']\
                                            .apply(lambda x: lgbm_user_items_dict.get(x, []))
    
    # cold start user file MP_list top25
    sumbission_cold = sumbission.copy()
    sumbission_cold['predicted_list'] = sumbission_cold['predicted_list']\
                                            .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])
    
    return sumbission, sumbission_cold

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')
sumbission_pred, sumbission_cold = evaluation(X_train, submission, n, MP_cand)

In [ ]:
set_list = {}
for list_i in sumbission_cold.predicted_list:
    set_list[str(list_i)] = 0

In [ ]:
print('같은 추천을 받은 유저 수:', 8311 - len(set_list))

In [ ]:
sumbission_cold

In [ ]:
# 제출 조건 충족 확인
assert submission.profile_id.nunique() == sumbission_cold.profile_id.nunique()
for pred_list in sumbission_cold.predicted_list:
    assert len(pred_list) == 25

In [ ]:
sumbission_cold.to_csv('lgbm_candidate_submission_NCF_candidate_ver10.csv', index=False)